In [1]:
import sys
sys.path.append('..\\src')

## Py4J or JPype

Demonstrate loading and querying data with CoreseAPI connected through `Py4J` or `JPype` packages.

### Connect to Corese API

In [2]:
python_to_java_bridge = 'py4j'

In [3]:
#%%timeit -n 1 -r 1
from  pycorese.api import CoreseAPI

corese = CoreseAPI(java_bridge=python_to_java_bridge)
corese.loadCorese()

2024-11-07 15:50:42,084 - INFO - Py4J: Loading CORESE...
2024-11-07 15:50:42,704 - INFO - Py4J: CORESE is loaded


### High-level API

#### Run SELECT query

In [13]:
import os 
data_path = os.path.abspath(os.path.join('data', 'beatles.rdf'))

query = '''
SELECT *
WHERE {?s ?p ?o} LIMIT 5'''

graph = corese.loadRDF(data_path) 
results = corese.sparqlSelect(graph, query=query, return_dataframe=True)

results.shape

(5, 3)

#### Load inference rules 

In [14]:
corese.resetRuleEngine(graph)
query = "select * where {?s a ?type} order by ?type"
print(corese.sparqlSelect(graph, query=query))
print("Graph size: ", graph.graphSize())


                                     s                           type
0  http://example.com/Please_Please_Me       http://example.com/Album
1         http://example.com/McCartney       http://example.com/Album
2           http://example.com/Imagine       http://example.com/Album
3       http://example.com/The_Beatles        http://example.com/Band
4       http://example.com/John_Lennon  http://example.com/SoloArtist
5    http://example.com/Paul_McCartney  http://example.com/SoloArtist
6       http://example.com/Ringo_Starr  http://example.com/SoloArtist
7   http://example.com/George_Harrison  http://example.com/SoloArtist
8        http://example.com/Love_Me_Do        http://example.com/Song
Graph size:  29


Adding inference rules to the Corese engine should change the results of the query by adding new triples.

In [17]:
corese.loadRuleEngine(graph, profile=corese.RuleEngine.Profile.RDFS)
print("Graph size: ", graph.graphSize())

Graph size:  33


Let's see what was added.

In [18]:
query = "select * where {?s a ?type} order by ?type"
print(corese.sparqlSelect(graph, query=query))
print("Graph size: ", graph.graphSize())

                                      s                           type
0   http://example.com/Please_Please_Me       http://example.com/Album
1          http://example.com/McCartney       http://example.com/Album
2            http://example.com/Imagine       http://example.com/Album
3        http://example.com/The_Beatles        http://example.com/Band
4        http://example.com/John_Lennon      http://example.com/Person
5     http://example.com/Paul_McCartney      http://example.com/Person
6        http://example.com/Ringo_Starr      http://example.com/Person
7    http://example.com/George_Harrison      http://example.com/Person
8        http://example.com/John_Lennon  http://example.com/SoloArtist
9     http://example.com/Paul_McCartney  http://example.com/SoloArtist
10       http://example.com/Ringo_Starr  http://example.com/SoloArtist
11   http://example.com/George_Harrison  http://example.com/SoloArtist
12        http://example.com/Love_Me_Do        http://example.com/Song
Graph 

The inference was that the solo artist is also a person although it was not explicitly stated in the data.

#### Run CONSTRUCT query

In [19]:
prefixes = '@prefix ex: <http://example.com/>'
contruct = '''CONSTRUCT {?Beatle a ex:BandMember }
              WHERE { ex:The_Beatles ex:member ?Beatle}'''

results = corese.sparqlConstruct(graph, prefixes=prefixes, query=contruct)

print(results)

<?xml version="1.0"?>
<rdf:RDF
  xmlns:ex='http://example.com/'
  xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>

  <ex:BandMember rdf:about='http://example.com/Ringo_Starr'>
  </ex:BandMember>

  <ex:BandMember rdf:about='http://example.com/John_Lennon'>
  </ex:BandMember>

  <ex:BandMember rdf:about='http://example.com/George_Harrison'>
  </ex:BandMember>

  <ex:BandMember rdf:about='http://example.com/Paul_McCartney'>
  </ex:BandMember>

</rdf:RDF>


By default, the CONSTRUCT query returns the RDF/XML format. For more concise format convert the results to Turtle.

In [20]:
ttl = corese.toTurtle(results)

print(ttl)


<http://example.com/George_Harrison> a <http://example.com/BandMember> .

<http://example.com/John_Lennon> a <http://example.com/BandMember> .

<http://example.com/Paul_McCartney> a <http://example.com/BandMember> .

<http://example.com/Ringo_Starr> a <http://example.com/BandMember> .




## Low-level API

Adding triples manually to the graph.

In [21]:
# Namespace
ex = "http://example.com/"

# Get the graph from either Graph or DataManager objects
graph = graph.getGraph() 

# Create and add statement: Edith Piaf is an Singer
new_album_IRI = graph.addResource(ex + "Help")
rdf_Type_Property = graph.addProperty(corese.Namespaces.RDF + 'type')
album_type_IRI = graph.addResource(ex + "Album")

graph.addEdge(new_album_IRI, rdf_Type_Property, album_type_IRI)

query = f'''@prefix ex: <{ex}>
            select * 
            where {{?s a ?type filter (?type = ex:Album) }} 
            order by ?type'''

exec = corese.QueryProcess.create(graph)

results = exec.query(query)

print(results)


01 ?s = <http://example.com/Please_Please_Me>; ?type = <http://example.com/Album>; 
02 ?s = <http://example.com/McCartney>; ?type = <http://example.com/Album>; 
03 ?s = <http://example.com/Imagine>; ?type = <http://example.com/Album>; 
04 ?s = <http://example.com/Help>; ?type = <http://example.com/Album>; 

